## NEOM region swale potential

Amgad, James, Josh

Mapping NEOM region for suitability of building features (such as swales) to capture surface/ sub-surface water, mainly to build up plant-available water in the soil.

Layers/ Factors:
1. Slope : Restricted to [0-10 degrees], gradual slopes are more desirable
2. Top Soil Types: Medium texture (loam/ silt) score highest, second is fine clays, and last is coarse sands for retaining plant-avaialble water
3. Runoff: Higher runoff is better, masked to values only above 1E-3 mm per grid box

In [2]:
import ee
import numpy as np

import sys
sys.path.insert(1, '../datasets/SMAP-USDA/')  # until we package this
import visualize as vis
import water_common as common
import normalize as norm
from statistics import mean
import folium
from folium import plugins

Loading BokehJS ...

In [3]:
ee.Initialize()

In [4]:
#importing slope variable, masking for gradual slopes
neom = ee.Geometry.BBox(34.52778, 27.72417, 35.96528, 29.14139)

slope_img = ee.Image('users/jamesmcc/merit_slope/merit_terrain_slope')
slope_mask = slope_img.lte(10).And(slope_img.gt(0))
slope_img_inv = slope_img.updateMask(slope_mask).pow(-1)

slope_img_scaled = norm.img_scale(slope_img_inv, area_of_interest = neom)

In [5]:
#soil types
soil_types = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")

In [6]:
#categorizing soil types at 0, 10, and 30 cm depths based on retaining plant-available water ability
top_soils = [5,7,8,10]
medium_soils = [2,4,6,9]
low_soils = [1,3,11,12]
soil_0 = soil_types.expression(
        "(b('b0') == 5) ? 1.0" +
        ": (b('b0') == 7) ? 1.0" +
        ": (b('b0') == 8) ? 1.0" +
        ": (b('b0') == 10) ? 1.0" +
        ": (b('b0') == 2) ? 0.7" +
        ": (b('b0') == 4) ? 0.7" +
        ": (b('b0') == 6) ? 0.7" +
        ": (b('b0') == 9) ? 0.7" +
        ": (b('b0') == 1) ? 0.4" +
        ": (b('b0') == 3) ? 0.4" +
        ": (b('b0') == 11) ? 0.4" +
        ": (b('b0') == 12) ? 0.4" +
        ": 0")

soil_10 = soil_types.expression(
        "(b('b10') == 5) ? 1.0" +
        ": (b('b10') == 7) ? 1.0" +
        ": (b('b10') == 8) ? 1.0" +
        ": (b('b10') == 10) ? 1.0" +
        ": (b('b10') == 2) ? 0.7" +
        ": (b('b10') == 4) ? 0.7" +
        ": (b('b10') == 6) ? 0.7" +
        ": (b('b10') == 9) ? 0.7" +
        ": (b('b10') == 1) ? 0.4" +
        ": (b('b10') == 3) ? 0.4" +
        ": (b('b10') == 11) ? 0.4" +
        ": (b('b10') == 12) ? 0.4" +
        ": 0")

soil_30 = soil_types.expression(
        "(b('b30') == 5) ? 1.0" +
        ": (b('b30') == 7) ? 1.0" +
        ": (b('b30') == 8) ? 1.0" +
        ": (b('b30') == 10) ? 1.0" +
        ": (b('b30') == 2) ? 0.7" +
        ": (b('b30') == 4) ? 0.7" +
        ": (b('b30') == 6) ? 0.7" +
        ": (b('b30') == 9) ? 0.7" +
        ": (b('b30') == 1) ? 0.4" +
        ": (b('b30') == 3) ? 0.4" +
        ": (b('b30') == 11) ? 0.4" +
        ": (b('b30') == 12) ? 0.4" +
        ": 0")

top_soils = soil_0.expression('top_soil + soil_10 + soil_30',
                             {'top_soil': soil_0.select('constant'),
                             'soil_10': soil_10.select('constant'),
                             'soil_30': soil_30.select('constant')})

In [7]:
#scaling top soils and bottom soils
top_soils_scaled = norm.img_scale(top_soils, area_of_interest= neom)

In [8]:
#importing runoff for January, masking for values higher than 1e-03 mm

month = 1

runoff = ee.ImageCollection('users/jamesmcc/era5_sfc_runoff_neom_climatology')

runoff_img = (
    runoff
    .filter(ee.Filter.eq('month', month))
    .select('surface_runoff').first())

runoff_mask = runoff_img.gte(1e-06)
runoff_update = runoff_img.updateMask(runoff_mask)

runoff_scaled = norm.img_scale(runoff_update, area_of_interest = neom)

In [57]:
#calculate sum of potential water storage from neom area over a year

runoff_a = runoff.reduce(reducer = ee.Reducer.sum())

runoff_b = runoff_a.reduceRegion(reducer= ee.Reducer.sum(), geometry = neom, scale = 100)

runoff_sum = runoff_b.getInfo()['surface_runoff_sum']*(100**2)   #multiply water depth of each grid by surface area to obtain potential volume of water

print(runoff_sum)

{'surface_runoff_sum': 15.638897075176653}
156388.97075176652


In [99]:
#Compiling score, printing score range

score = ((runoff_scaled.multiply(.5))
        .add(top_soils_scaled.multiply(.25))
        .add(slope_img_scaled.multiply(.25)))
score_range = norm.img_range(score, area_of_interest = neom)
print(score_range)

[0.0018684243936366606, 0.7113979901770086]


In [100]:
#visualization parameters

palette_name = 'Blues'
palette_len = 4
palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(palette=palette, minimum = 0.1, maximum= 0.5)

vis_params = {
    'min': 0.1, 'max': 0.5,'dimensions': 512,
    'palette': palette}

vis_params_var = {
    'min': 0, 'max': 1,'dimensions': 512,
    'palette': palette}

In [101]:
#plotting score for NEOM region

the_map = vis.folium_map(location=[28.5, 35.3], zoom_start=8, height=500)
the_map.add_ee_layer(score, vis_params, name = 'Score')
vis.folium_display(the_map)